In [1]:
import psycopg2
import pandas as pd

In [ ]:
# подтягивание аккаунта из excel

profile = pd.read_excel("account.profile.xlsx").astype(str)
profile['profile_tg_id'].astype(str)
profile['profile_role_name'].astype(str)
profile['profile_name'].astype(str)


# загрузка в бд schedule.lesson

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in profile.iterrows():
    try:
        cur.execute("""
            INSERT INTO account.profile (profile_tg_id, profile_role_name, profile_name)
            VALUES (%s, %s, %s)
            """,
            (
                row['profile_tg_id'],
                row['profile_role_name'],
                row['profile_name'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

profile

In [ ]:
# подтягиваем информацию о расписании из excel

final_res = pd.DataFrame()
for i in range(4):
    exec(f'lessons = pd.read_excel("1 курс_группы_11ТМ_12ИИ_13ИИ_14БТ.xlsx", sheet_name = {i})')
    x = 0 
    for j in range(len(lessons.columns)//13):  # type: ignore
        exec(f'df_{j} = lessons[lessons.columns[{x}:{x+13}].tolist()]')
        exec(f'df_{j}.columns = lessons.columns[:13].tolist()')
        x+=14
    print(i)
    df = pd.concat([df_0,df_1,df_2,df_3,df_4]) # type: ignore
    df = df.dropna(how='all')
    df[df.columns[0]] = df[df.columns[0]].fillna(method='ffill')

    result = pd.DataFrame()

    for c in range(1,len(df.columns),3):
        df_copy = df[[df.columns[0],df.columns[c],df.columns[c+1],df.columns[c+2]]]
        df_copy[df.columns[c+1]] = df_copy[df.columns[c+1]].fillna(method='ffill')
        df_date_sub_copy = df_copy.loc[(df_copy[df.columns[c+2]].isna())][[df.columns[0],df.columns[c],df.columns[c+1]]]
        df_copy= df_copy.dropna()
        exec(f'res_{c} = df_copy.merge(df_date_sub_copy, on =[df.columns[0],df.columns[{c}+1]])')
        exec(f'res_{c}["group"] = df.columns[{c}]')
        exec(f'res_{c}.columns = ["Дата занятия","Тип занятия","Время занятия","Аудитория","Предмет","Группа"]')
        exec(f'result = pd.concat([result, res_{c}]).reset_index(drop=True)')
        exec(f'result.dropna(inplace=True)')
    final_res = pd.concat([final_res,result]).reset_index(drop=True)    
final_res['Дата занятия']=final_res['Дата занятия'].str[:-3] + ".2024"


final_res['Время начала занятия'] = final_res['Время занятия'].str.split('\n', expand=True)[0].str.replace('-', ':')
final_res['Время конца занятия'] = final_res['Время занятия'].str.split('\n', expand=True)[1].str.replace('-', ':')
final_res['ссылка'] = 'null'
final_res['Дата занятия'] = pd.to_datetime(final_res['Дата занятия'], format='%d.%m.%Y')
final_res['Время начала занятия'] = final_res['Дата занятия'].astype(str) + ' ' + final_res['Время начала занятия'].astype(str)
final_res['Время конца занятия'] = final_res['Дата занятия'].astype(str) + ' ' + final_res['Время конца занятия'].astype(str)
final_res['id_учителя']='0'

final_res = final_res[['Группа', 'Тип занятия', 'Аудитория', 'ссылка', 'Время начала занятия', 'Время конца занятия', 'id_учителя', 'Предмет']]

final_res['Тип занятия'] = final_res['Тип занятия'].replace(['Лекции'], 'Лекция')
final_res['Тип занятия'] = final_res['Тип занятия'].replace(['Практические занятия'], 'Практическое занятие')
final_res['Тип занятия'] = final_res['Тип занятия'].replace(['Лабораторные занятия'], 'Лабораторное занятие')

# задаем к каждому предмету id препода

final_res.loc[final_res['Предмет']== 'Технологии AR/VR',                                                    ['id_учителя'] ]= '123456789'
final_res.loc[final_res['Предмет']== '3D анатомия - физиология',                                            ['id_учителя'] ]= '987654321'
final_res.loc[final_res['Предмет']== 'Ознакомительная практика',                                            ['id_учителя'] ]= '234567890'
final_res.loc[final_res['Предмет']== 'Медицинская статистика',                                              ['id_учителя'] ]= '345678901'
final_res.loc[final_res['Предмет']== 'Анализ данных на  Python',                                            ['id_учителя'] ]= '456789012'
final_res.loc[final_res['Предмет']== 'Медицинские аппаратно-программные комплексы',                         ['id_учителя'] ]= '567890123'
final_res.loc[final_res['Предмет']== 'Системный анализ и организация здравоохранения',                      ['id_учителя'] ]= '678901234'
final_res.loc[final_res['Предмет']== 'Методы глубокого обучения',                                           ['id_учителя'] ]= '789012345'
final_res.loc[final_res['Предмет']== 'Системы поддержки принятия решений',                                  ['id_учителя'] ]= '890123456'
final_res.loc[final_res['Предмет']== 'Иностранный язык в профессиональной сфере',                           ['id_учителя'] ]= '901234567'
final_res.loc[final_res['Предмет']== 'Основы молекулярной биологии',                                        ['id_учителя'] ]= '112233445'
final_res.loc[final_res['Предмет']== 'Педагогическая практика',                                             ['id_учителя'] ]= '223344556'
final_res.loc[final_res['Предмет']== 'Базовые аспекты биохимии человека',                                   ['id_учителя'] ]= '334455667'
final_res.loc[final_res['Предмет']== 'Методическое сопровождение применения диагностических тест-систем',   ['id_учителя'] ]= '445566778'
final_res.loc[final_res['Предмет']== 'Биоинформатика',                                                      ['id_учителя'] ]= '556677889'
final_res.loc[final_res['Предмет']== 'Основы медицинской микробиологии и иммунологии',                      ['id_учителя'] ]= '667788990'
final_res.loc[final_res['Предмет']== 'Технологическая акселерация',                                         ['id_учителя'] ]= '778899001'


# загрузка в бд schedule.lesson

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in final_res.iterrows():
    try:
        cur.execute("""
            INSERT INTO schedule.lesson (lesson_type_group_name, lesson_type_education_name, lesson_type_education_room_name,
            lesson_type_education_room_link, lesson_date_time_start, lesson_date_time_end,
            lesson_link, lesson_teacher_id, lesson_subject_name)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (
                row['Группа'],
                row['Тип занятия'],
                row['Аудитория'],
                row['ссылка'],
                row['Время начала занятия'],
                row['Время конца занятия'],
                row['ссылка'],
                row['id_учителя'],
                row['Предмет']
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

final_res

In [ ]:
final_res['Предмет'].unique()

In [ ]:
# получение кафедр

type_department = pd.read_excel("teacher_info.type_department.xlsx").astype(str)
type_department['кафедра'].astype(str)


# загрезка кафедр в бд teacher_info.type_department

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in type_department.iterrows():
    try:
        cur.execute("""
            INSERT INTO teacher_info.type_department (type_department_name)
            VALUES (%s)
            """,
            (
                row['кафедра'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

type_department

In [ ]:

teacher = pd.read_excel("teacher_info.teacher.xlsx")


# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in teacher.iterrows():
    try:
        cur.execute("""
            INSERT INTO teacher_info.teacher (teacher_profile_tg_id, teacher_name, teacher_job, teacher_department, teacher_adress, teacher_email)
            VALUES (%s, %s, %s, %s, %s, %s)
            """,
            (
                row['тг айди'],
                row['фио препода'],
                row['должность'],
                row['кафедра'],
                row['адрес'],
                row['рабочая почта'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

teacher

In [ ]:

teacher_img = pd.read_excel("teacher_info.image.xlsx")


# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in teacher_img.iterrows():
    try:
        cur.execute("""
            INSERT INTO teacher_info.image (teacher_id, name_img)
            VALUES (%s, %s)
            """,
            (
                row['teacher_id'],
                row['name_img'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

teacher_img

In [ ]:

adress_contacts_type_place = pd.read_excel("adress_contacts.type_place.xlsx")

# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in adress_contacts_type_place.iterrows():
    try:
        cur.execute("""
            INSERT INTO adress_contacts.type_place (type_place_name)
            VALUES (%s)
            """,
            (
                row['type_place_name'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

adress_contacts_type_place

In [ ]:
adress_contacts_place = pd.read_excel("adress_contacts.place.xlsx").astype(str)
adress_contacts_place['place_latitude'] = adress_contacts_place['place_latitude'].astype(float)
adress_contacts_place['place_longitude'] = adress_contacts_place['place_longitude'].astype(float)

# загрезка кафедр в бд teacher_info.teacher


conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in adress_contacts_place.iterrows():
    try:
        cur.execute("""
            INSERT INTO adress_contacts.place (place_name, place_time_start, place_time_end, place_phone, place_email, place_adress, place_latitude, place_longitude)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (
                row['place_name'],
                row['place_time_start'],
                row['place_time_end'],
                row['place_phone'],
                row['place_email'],
                row['place_adress'],
                row['place_latitude'],
                row['place_longitude'],
            )
        )
        
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

adress_contacts_place

In [ ]:
adress_contacts_place_info = pd.read_excel("adress_contacts.place_info.xlsx")

# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in adress_contacts_place_info.iterrows():
    try:
        cur.execute("""
            INSERT INTO adress_contacts.place_info (place_info_type_place_name, place_info_place_name)
            VALUES (%s, %s)
            """,
            (
                row['place_info_type_place_name'],
                row['place_info_place_name'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

adress_contacts_place_info

In [ ]:
documents_type_group = pd.read_excel("documents.type_group.xlsx")

# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in documents_type_group.iterrows():
    try:
        cur.execute("""
            INSERT INTO documents.type_group (type_group_name)
            VALUES (%s)
            """,
            (
                row['type_group_name'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

documents_type_group

In [ ]:
documents_type_file = pd.read_excel("documents.type_file.xlsx")

# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in documents_type_file.iterrows():
    try:
        cur.execute("""
            INSERT INTO documents.type_file (type_file_name)
            VALUES (%s)
            """,
            (
                row['type_file_name'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

documents_type_file

In [ ]:
documents_document = pd.read_excel("documents.document.xlsx")

# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in documents_document.iterrows():
    try:
        cur.execute("""
            INSERT INTO documents.document (file_group, file_name)
            VALUES (%s, %s)
            """,
            (
                row['file_name'],
                row['file_group'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

documents_document